In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T
from snowflake.snowpark import Window

In [ ]:
table_name = st.selectbox(
    label="Table Selector",
    options=[x["name"] for x in session.sql("show tables").collect()]
)

In [ ]:
select * from {{table_name}} limit 100

In [ ]:
df = cells.cell4.to_df()
if table_name != "JSON_TABLE":
    df.show()
else:
    df.show(1)

In [ ]:
df = session.table("json_table")
df.show(1)

In [ ]:
df.select(
    F.col("v"),
    # keys ARE case sensitive!
    F.col("v")["BIKE"]
).show(1)

In [ ]:
df.select(
    # keys ARE case sensitive!
    F.col("v")["BIKE"],
    F.col("v")["BIKE"]["BIKEID"],
    F.col("v")["RIDER"]
).show(1)

In [ ]:
df.select(
    # keys ARE case sensitive!
    F.col("v")["BIKE"].alias("bike_info"),
    F.col("v")["BIKE"]["BIKEID"].alias("bike_id"),
    F.col("v")["RIDER"].alias("rider_info")
).show(1)

In [ ]:
df.select(
    F.to_date(F.col("v")["STARTTIME"]).alias("start_date"),
    F.to_date(F.col("v")["ENDTIME"]).alias("end_date")
).show()

In [ ]:
weather = session.table("weather")
weather.show(5)

In [ ]:
weather.select(F.col("postal_code")).distinct()

In [ ]:
# All NY-Metro zip codes
# Aggregate into itself to represent the NY-Metro

weather = weather.select(
    F.lit("NY Metro").alias("state"),
    F.col("postal_code"),
    F.col("date_valid_std").alias("observation_date"),
    F.col("min_temperature_air_2m_f"),
    F.col("max_temperature_air_2m_f"),
    F.col("avg_temperature_air_2m_f"),
    F.col("tot_precipitation_in"),
    F.col("tot_snowfall_in"),
    F.col("tot_snowdepth_in"),
    F.col("avg_wind_speed_100m_mph")
).group_by(
    [
        F.col("state"), F.col("observation_date")
    ]
).agg(
        F.avg(F.col("min_temperature_air_2m_f")).alias("min_temp"),
        F.avg(F.col("max_temperature_air_2m_f")).alias("max_temp"),
        F.avg(F.col("avg_temperature_air_2m_f")).alias("avg_temp"),
        F.avg(F.col("tot_precipitation_in")).alias("total_precipitation"),
        F.avg(F.col("tot_snowfall_in")).alias("total_snowfall"),
        F.avg(F.col("tot_snowdepth_in")).alias("snowdepth"),
        F.avg(F.col("avg_wind_speed_100m_mph")).alias("wind_speed")
    )

In [ ]:
weather.show()

In [ ]:
weather.count()

In [ ]:
weather.select(
    F.max(F.col("observation_date")).alias("max_date"),
    F.min(F.col("observation_date")).alias("min_date")
).show()

In [ ]:
ads = session.table("advertisements")
ads.show()

In [ ]:
ads.join(
    weather,
    F.to_date(ads["starttime"])==weather["observation_date"]
).show()

In [ ]:
kiosks = session.table("kiosks")
kiosks.show(1)

In [ ]:
ads.join(
    kiosks,
    ads["start_station_id"]==kiosks["station_id"]
).show(2)

In [ ]:
ad_locations = ads.join(
    kiosks,
    ads["start_station_id"]==kiosks["station_id"]
)


In [ ]:
ad_locations.select(
    *ad_locations,
    F.to_date(F.col("starttime")).alias("date"),
    # Time of Day feature
    # 3 - Late / 0 - Morning / 1 - Afternoon / 2 - Evening
    (
        F.when(
            F.to_time(F.col("starttime")).between('00:00', '04:59'),
            F.lit(3)
        ).when(
            F.to_time(F.col("starttime")).between('05:00', '11:59'),
            F.lit(0)
        ).when(
            F.to_time(F.col("starttime")).between('12:00', '16:59'),
            F.lit(1)
        ).when(
            F.to_time(F.col("starttime")).between('17:00', '23:59'),
            F.lit(2)
        )
    ).alias("time_of_day")
    ).select(
        F.col("date"),
        F.col("time_of_day"),
        F.col("station_id")
    ).groupBy(
        F.col("date"),
        F.col("time_of_day"),
        F.col("station_id")
    ).count().select(
        F.col("date"),
        F.col("time_of_day"),
        F.col("station_id"),
        F.col("count").alias("y"),
        F.log(
            base=10,
            x=F.col("count")
        ).alias("y_log"),
        F.lag(
            F.col("y_log")).over(
                window=Window.orderBy(
                    [
                        F.col("station_id"), F.col("time_of_day"), F.col("date")
                    ]
                )
            ).alias("y_log_lag")
    ).orderBy(
        F.col("date").asc(),
        F.col("time_of_day").asc(),
        F.col("station_id")
    ).show()

In [ ]:
ad_locations = ad_locations.select(
    *ad_locations,
    F.to_date(F.col("starttime")).alias("date"),
    # Time of Day feature
    # 3 - Late / 0 - Morning / 1 - Afternoon / 2 - Evening
    (
        F.when(
            F.to_time(F.col("starttime")).between('00:00', '04:59'),
            F.lit(3)
        ).when(
            F.to_time(F.col("starttime")).between('05:00', '11:59'),
            F.lit(0)
        ).when(
            F.to_time(F.col("starttime")).between('12:00', '16:59'),
            F.lit(1)
        ).when(
            F.to_time(F.col("starttime")).between('17:00', '23:59'),
            F.lit(2)
        )
    ).alias("time_of_day")
    ).select(
        F.col("date"),
        F.col("time_of_day"),
        F.col("station_id")
    ).groupBy(
        F.col("date"),
        F.col("time_of_day"),
        F.col("station_id")
    ).count().select(
        F.col("date"),
        F.col("time_of_day"),
        F.col("station_id"),
        F.col("count").alias("y"),
        F.log(
            base=10,
            x=F.col("count")
        ).alias("y_log"),
        F.lag(
            F.col("y_log")).over(
                window=Window.orderBy(
                    [
                        F.col("station_id"), F.col("time_of_day"), F.col("date")
                    ]
                )
            ).alias("y_log_lag")
    ).orderBy(
        F.col("date").asc(),
        F.col("time_of_day").asc(),
        F.col("station_id")
    )

In [ ]:
ad_locations.join(
    weather,
    ad_locations["date"] == weather["observation_date"]
).show()

In [ ]:
ad_locations_weather = ad_locations.join(
    weather,
    ad_locations["date"] == weather["observation_date"]
)

ad_locations_weather.write.save_as_table(table_name="david_table", mode="overwrite")